## Assumptions and Constants
Let's use th metal ABR core because it sounds, well, metal.

In [90]:
# Reactor constants
beta_eff = 0.00334
prompt_lifetime = 0.38

Na void positive reactivity due to spectral hardening

Less fast neutrons, less absorption (?)

![image](img/Pu_239_cross_section.png)

## Operative word is "Local"

* Void worth $4.44 assuming "flowing sodium completely voided in ALL active and above-core regions" ([Argonne's Fast Reactor Physics - 2](https://www.nrc.gov/docs/ML1914/ML19148A801.pdf))
* How to apply this for local voiding?
* Ultimately want reactivty as a function of sodium density, i.e. $k(\rho_{Na})$

## Driven by scattering out of fast spectrum
Assumptions to derive $k(\rho_{Na})$
* Assume two group equation
* Infinite reactor (or perfect reflector)
* All relevant fissions occur in fast spectrum (terrible assumption...)
* Everything scattered out of fast spectrum is absorbed (also pretty bad)

## Two group equation
Group 1 - fast spectrum

Group 2 - everything else
$$
\frac{1}{k}\nu_1 \Sigma_f \phi_1 - \Sigma_{a1} \phi_1 - \Sigma_{1\rightarrow 2}\phi_1 = 0 \\
\Sigma_{1 \rightarrow 2} \phi_1 - \Sigma_{a2} \phi_2  = 0
$$

Solving for k in the group 1 equation
$$
k = \frac{\nu_1 \Sigma_{f1}}{\Sigma_{a1} + \Sigma_{1 \rightarrow 2}} 
$$

Using group 2 to make a statement about $\Sigma_{a1}$, noting that $\Sigma_{a2} \propto \Sigma_{a1}$
$$
\Sigma_{a2}\phi_2 = \Sigma_{1 \rightarrow 2} \phi_1 = C_a \Sigma_{a1}\phi1\\
\Sigma_{a1} = \frac{\Sigma_{1 \rightarrow 2} \phi_1}{C_a \phi_2}
$$
where $C_a$ is a proportionality constant.

Substituting $\Sigma_{a1}$ into the expression for k

$$
k = \frac{\nu_1 \Sigma_{f1}}{\frac{\Sigma_{1 \rightarrow 2} \phi_1}{C_a \phi_2} + \Sigma_{1 \rightarrow 2}}
= \frac{\nu_1 \Sigma_{f1}}{\Sigma_{1 \rightarrow 2}[\frac{\phi_1}{C_a \phi_2} + 1]}

= \frac{\nu_1\Sigma_{f1}}{\frac{\rho_{Na}A_v}{M_{Na}}\sigma_{1 \rightarrow 2}[\frac{\phi_1}{C_a \phi_2} + 1]}
$$


$$
k = \frac{\nu_1 \Sigma_{f1}}{\Sigma_{a1} + \Sigma_{1 \rightarrow 2}}

= \frac{\nu_1\Sigma_{f1}}{\Sigma_{a1} + \frac{\rho_{Na}A_v}{M_{Na}}\sigma_{1 \rightarrow 2}}

= \frac{A}{B + C\rho_{Na}}
$$



$$
k(\Sigma_{a1} + \Sigma_{1 \rightarrow 2}) = \nu_1 \Sigma_{f1}
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\Sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\rho_{Na}A_v}{M_{Na}}\frac{\sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
\begin{equation}
k\left(A + B\rho_{Na}\right) = 1 
\end{equation}
$$


We have two known reactor conditions that we can now use to solve for A and B in the above equation:
1. Void worth when entire core is voided ($4.44 BOC, metal, startup)
2. Assume k = 1 when entire core is liquid

For Na density, we will use (from [ANL-RE-95-2](https://www.ne.anl.gov/eda/ANL-RE-95-2.pdf))
1. Density of vapor sodium at saturation ($\sim 1200 K$ at operating pressure from [Figure III.7-12](https://publications.anl.gov/anlpubs/2017/04/134264.pdf)) is $0.394 \frac{kg}{m^3}$.
2. Density of liquid sodium at core average temperature $700 K$ (average of $350^o C$ cold pool temperature and $500^o C$ hot temperature) is $852 \frac{kg}{m^3}$.

This results in a 2x2 linear system that we can use to solve for A and B:
$$
\begin{align} \tag{2}
Ak_g + Bk_g\rho_{na,g} = 1 \\
Ak_l + Bk_l\rho_{na,l} = 1
\end{align}
$$

$$
\begin{bmatrix}
k_g & k_g \rho_{na,g} \\
k_l & k_l \rho_{na,l} \\
\end{bmatrix}

\begin{bmatrix}
A \\ B
\end{bmatrix} 

=

\begin{bmatrix}
1 \\ 1
\end{bmatrix}
$$

It's a little confusing, but don't forget $\rho$ in these equations is Na density, not reactivity. Now for some python! (Finally...)

In [91]:
"""A few helper functions before we get started.

And a general note on comments since you'll want to know how I
work. My rule of thumb is to write comments according to my audience.
In many cases, my audience is engineers, and therefore my comments
are extensive. But comments, like lines of code, represent inventory
of an application. As your inventory goes up, your maintenance costs
increase as well. Comments require maintenance just like source code
in the case of refactoring. If the code is obvious or the audience is
savvy, comments oftentimes are not necessary. However, I'll be using
lots of comments in this project assuming my audience is primarily engineers.
Also computational code is confusing and hard to read.

Most of the code I write is in standalone applications rather than Jupyter
notebooks, so I'll use google's docstring guide inside of some code blocks
where appropriate (which is sort of what you see here).

I will also use "_" to demark "private" values or functions that are local
to a computation and won't be used elsewhere (and therefore won't be 
documented in docstrings like this).

Attributes:
    ureg (pint.UnitRegistry): Pint unit registry
    Quantity (pint.Quantity): Pint float quantity containing units
"""
import pint
from math import floor, log10

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()
Quant = pint.Quantity

def round_quant(quantity, sig_fig=4):
    """Return pint quantity reduced to given significant digits as a string."""
    value = round(
        quantity.m, 
        sig_fig-int(floor(log10(abs(quantity.m)))) - 1
    )
    return str(value) + ' ' + str(quantity.units)


In [92]:
"""Solving for A and B in equation 1.

Assumes metal startup core at BOC. k_1, rho_1, k_2, and rho_2
from Equation 2 are first determined using the assumptions 
outlined above. They become the coefficients for the 2x2 system
used to solve for A and B.

Attributes:
    global_void_reactivity (float): Whole core void reactivity
    k_g (float): Multiplication factor with entire core voided
    k_l (float): Reactor critical multiplication factor or 1.0
    rho_na_g (Quantity): Vapor density of sodium
    rho_na_l (Quantity): Liquid density of sodium
    A (float): Constant from Equation 1
    B (float): Constant from Equation 1
"""
import numpy as np

# k_g from Equations 2 is solved from the void worth 
# of the entire core
void_worth = 4.44 # Dollars
beta_eff = 0.0034 # Delayed neutron fraction for metal core
global_void_reactivity = void_worth * beta_eff
k_g = 1 / (1 - global_void_reactivity)
# k2 is assumed to be 1
k_l = 1.0

# Density of voided sodium
rho_na_g = Quant(0.394, "kg/m^3")
# Density of liquid sodium
rho_na_l = Quant(852, "kg/m^3")

# Solving the 2x2 system for coefficients A and B in Equation 1
# 2x2 coefficient matrix
coeffs = np.array([[k_g, k_g*rho_na_g.m], [k_l, k_l*rho_na_l.m]])
# Right hand side term
rhs = np.array([1, 1])
# Solving for constants A and B from Equation 1
A,B = np.linalg.solve(coeffs, rhs)

Now we can return reactivity as a function of core sodium void fraction. 
This is my interpretation of "Local" in question 1, part 1 for the miniproject.
A subchannel code (e.g. NUPAC) could be used to generate this local void fraction.
As it is I've spent a lot of time on this (fairly simple) derivation and won't try and build a subchannel solver right now :).

In [93]:
def void_reactivity(alpha_na):
    """Calculate the reactivity insertion of ABR due to Na voiding.
    
    Using the void worth of the active whole-core, calculate the
    reactivity insertion from a fraction of void within a single
    assembly using the following relationships:
        * reactivity = k - 1 / k
        * k = 1 / (A + B*rho_na) from Equation 1
        * rho_na = alpha_na*rho_na_g + (1- alpha_na)*rho_na_l 

    Args:
        alpha_na (float): Sodium void fraction of ABR core. 
            1.0 = Entire active core is vapor
            0.0 = Enteire active core is liquid

    Returns:
        float: Reactivity insertion as a function of Na void fraction
    """
    # This function is only valid for alpha_na between 1 and 0, so 
    # lets do some raisen if this isn't the case.
    if alpha_na > 1.0 or alpha_na < 0.0:
        raise ValueError(
            "Sodium volume fraction must be between 1.0 and 0.0. "
            f"Received a value of {alpha_na}."
        )

    # Core sodium density
    rho_na = alpha_na*rho_na_g + (1-alpha_na)*rho_na_l

    # Multiplication factor
    k = 1 / (A + rho_na.m*B)

    # Return reactivity
    return (k-1)/k
    


In [96]:
"""Test void reactivity equaiton.

This of course goes in a test module, but I figured it would
be good to have it here as well. Assertions are based on the
assumptions for a voided core and a liquid core.
"""
from glob import glob
import pytest

# Make sure exceptions are handled appropriately
with pytest.raises(ValueError) as err:
    # The Answer s way too big!
    void_reactivity(42)
# I always check my error statements since you could raise for
# the wrong reason.
assert "Received a value of 42" in str(err.value)

with pytest.raises(ValueError) as err:
    # Negative The Answer?
    void_reactivity(-42)
assert "Received a value of -42" in str(err.value)

# Check for liquid solid core
assert void_reactivity(0.0) == 0.0
# Check for vapor solid core
assert void_reactivity(1.0) == pytest.approx(global_void_reactivity)

It's a bummer you don't get green dots from assertions in Jupyter notebooks like with pytest. I think I'm addicted to those things.

To get an expression for $\rho_{Na}$ we'll need some geomtry information, and more assumptions of course.

In [97]:
import pint
import numpy as np
from math import log10, floor

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()

# Calculate the total volume of an assembly for 217 hexagonal assembly
# with 8 pins along each side, only considering the active core region
pins_per_side = 8
pitch = 16.142 * ureg.cm
assembly_side = pitch
active_core_height = 81.3 * ureg.cm
assembly_volume = float(3*np.sqrt(3)/2)*assembly_side**2 * active_core_height

# Calculate the total volume of fuel pins in an assembly
total_pins = 217
pin_diameter = 0.755 * ureg.cm
pin_volume = 3.1415 * (pin_diameter/2)**2 * active_core_height
total_pin_volume = pin_volume * total_pins

# Calculate the sodium volume in the active core region
sodium_volume = assembly_volume - total_pin_volume

print(
    "Assembly volume in active core region: ", 
    round_quant(assembly_volume, 4)
)
print(
    "Total pin volume in active core region: ", 
    round_quant(total_pin_volume, 4)
)
print(
    "Sodium volume in active core region: ", 
    round_quant(sodium_volume, 4)
)

Assembly volume in active core region:  55040.0 centimeter ** 3
Total pin volume in active core region:  7898.0 centimeter ** 3
Sodium volume in active core region:  47140.0 centimeter ** 3
